In [24]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import sklearn

# Data Exploration

In [25]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(r"C:\Users\maxso\PycharmProjects\ApartmentScraper\booli\scrapy_apartments.db")
df = pd.read_sql_query("SELECT * from apartment", con)


# Verify that result of SQL query is stored in the dataframe
print(df.head().T)

                                 0                 1  \
id                 /annons/3848254    /bostad/280424   
type                      Lägenhet          Lägenhet   
city                       Uppsala       Helsingborg   
street                   Torngatan  Lilla Bergaliden   
street_num                      24                 1   
district              Kapellgärdet           Centrum   
postal_code                  75423             25223   
latitude                   59.8664           56.0449   
longitude                  17.6387           12.7005   
rooms                            2                 2   
square_meters                   55                74   
floor                            1                 2   
housing_society       BRF Tornet 1    BRF Minerva 16   
construction_year             2011              1939   
fee                           3126              5028   
price                    2.095e+06         2.175e+06   

                                               

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6911 entries, 0 to 6910
Data columns (total 16 columns):
id                   6911 non-null object
type                 6911 non-null object
city                 6911 non-null object
street               6805 non-null object
street_num           6805 non-null object
district             6911 non-null object
postal_code          6911 non-null int64
latitude             6911 non-null float64
longitude            6911 non-null float64
rooms                6911 non-null float64
square_meters        6911 non-null int64
floor                6911 non-null int64
housing_society      5909 non-null object
construction_year    6096 non-null float64
fee                  6086 non-null float64
price                6578 non-null float64
dtypes: float64(6), int64(3), object(7)
memory usage: 864.0+ KB


In [27]:
print(df.describe())

        postal_code     latitude    longitude        rooms  square_meters  \
count   6911.000000  6911.000000  6911.000000  6911.000000    6911.000000   
mean   34252.778469    58.438192    15.793085     2.742295      72.000434   
std    23931.217763     1.453051     2.498384     1.507556      41.477190   
min    11121.000000    55.510615    11.710321     1.000000       6.000000   
25%    12473.000000    57.699510    12.982253     2.000000      44.000000   
50%    21633.000000    59.294872    17.617292     2.000000      62.000000   
75%    58232.000000    59.348830    18.013328     3.000000      87.000000   
max    75758.000000    60.146462    18.391504    18.000000     430.000000   

            floor  construction_year           fee         price  
count  6911.00000        6096.000000   6086.000000  6.578000e+03  
mean      2.47012        1968.389928   3545.184850  3.423673e+06  
std       2.72963          39.190492   1414.889842  3.086944e+06  
min       0.00000        1643.000000  

In [28]:
df.isnull().sum()

id                      0
type                    0
city                    0
street                106
street_num            106
district                0
postal_code             0
latitude                0
longitude               0
rooms                   0
square_meters           0
floor                   0
housing_society      1002
construction_year     815
fee                   825
price                 333
dtype: int64

In [29]:
df.isnull().sum()/df.count() * 100

id                    0.000000
type                  0.000000
city                  0.000000
street                1.557678
street_num            1.557678
district              0.000000
postal_code           0.000000
latitude              0.000000
longitude             0.000000
rooms                 0.000000
square_meters         0.000000
floor                 0.000000
housing_society      16.957184
construction_year    13.369423
fee                  13.555702
price                 5.062329
dtype: float64

we can see that around 5%, or 333 rows have missing values for the target variable. We can take these aside at a later point and, given that we are successful in creating an adequate model, use this model to predict these values. These rows can then be used to re-train a new model on. 

We can also see that almost 1.6% of *street* and *street_num* are missing. However, since *postal_code* technically contains roughly the same information, this will probably not be cause of any concern. We might even consider dropping these rows entirely.

Around 17% of the *housing_society* values are missing. This is what we will focus on exploring next.

## Housing Society exploration

In [30]:
df[df.housing_society.isnull()].head(50)

,id,type,city,street,street_num,district,postal_code,latitude,longitude,rooms,square_meters,floor,housing_society,construction_year,fee,price
6,/bostad/1846982,Villa,Helsingborg,Rusthållsgatan,5A,Miatorp,25231,56.015764,12.725562,5.0,112,0,None,1985.0,NaN,3295000.0
7,/bostad/2247046,Villa,Helsingborg,Grännagatan,7,Fältabacken,25250,56.039595,12.727825,4.0,95,0,None,1924.0,NaN,2995000.0
10,/bostad/2242476,Villa,Helsingborg,Alvägen,13,Laröd,25482,56.093354,12.647752,8.0,161,0,None,1965.0,NaN,7475000.0
33,/annons/3887215,Villa,Västerås,Fältspatsvägen,1,Munga,72594,59.742667,16.526091,2.0,55,0,None,1973.0,NaN,1995000.0
36,/bostad/2073765,Villa,Västerås,Malörtsvägen,16,Örtagården,72591,59.564192,16.465578,6.0,170,0,None,2003.0,NaN,5850000.0
37,/annons/3891158,Parhus,Helsingborg,Rååvägen,60B,Råå,25270,56.001603,12.737404,5.0,177,0,None,2020.0,NaN,6495000.0
51,/bostad/2815284,Villa,Helsingborg,Långgatan,17,Alla Bilder,25270,55.997960,12.739719,3.0,80,0,None,NaN,NaN,3475000.0
52,/annons/3877132,Villa,Helsingborg,Lövstigen,1,Miatorp,25241,56.023710,12.711099,7.0,167,0,None,1923.0,NaN,3995000.0
53,/annons/3877528,Villa,Helsingborg,Färjemansgatan,32,Tågaborg,25440,56.062210,12.690828,9.0,250,0,None,1923.0,NaN,9200000.0
68,/bostad/2805939,Villa,Helsingborg,Brännerigatan,4A,Ödåkra,25474,56.105326,12.744431,10.0,297,0,None,1929.0,NaN,4495000.0


It looks like it's mostly different kinds of houses such as villas and terrace houses that are missing *house_society*, which is not very surprising. It is mainly *Lägenheter*/apartments that are tied to housing societies.

In [31]:
df.groupby("type").count()

,id,city,street,street_num,district,postal_code,latitude,longitude,rooms,square_meters,floor,housing_society,construction_year,fee,price
type,,,,,,,,,,,,,,,
Fritidshus,104,104,101,101,104,104,104,104,104,104,104,1,82,3,104
Gård,9,9,8,8,9,9,9,9,9,9,9,0,9,0,8
Kedjehus,40,40,39,39,40,40,40,40,40,40,40,4,39,5,38
Lägenhet,5990,5990,5913,5913,5990,5990,5990,5990,5990,5990,5990,5817,5245,5957,5677
Parhus,53,53,49,49,53,53,53,53,53,53,53,24,46,33,49
Radhus,191,191,187,187,191,191,191,191,191,191,191,61,186,85,189
Villa,524,524,508,508,524,524,524,524,524,524,524,2,489,3,513


In [32]:
#num of null-values per type
df.housing_society.isnull().groupby(df["type"]).sum()

type
Fritidshus    103.0
Gård            9.0
Kedjehus       36.0
Lägenhet      173.0
Parhus         29.0
Radhus        130.0
Villa         522.0
Name: housing_society, dtype: float64

In [33]:
#percentage of null-values per type
df.housing_society.isnull().groupby(df["type"]).mean() * 100

type
Fritidshus     99.038462
Gård          100.000000
Kedjehus       90.000000
Lägenhet        2.888147
Parhus         54.716981
Radhus         68.062827
Villa          99.618321
Name: housing_society, dtype: float64

In [34]:
df[(df["type"] == 'Lägenhet') & (df["housing_society"].isnull())]

,id,type,city,street,street_num,district,postal_code,latitude,longitude,rooms,square_meters,floor,housing_society,construction_year,fee,price
93,/annons/3645502,Lägenhet,Västerås,Notuddsallén,7,Öster Mälarstrand,72358,59.605221,16.570524,3.0,88,2,None,2018.0,4840.0,3144000.0
107,/annons/3782832,Lägenhet,Västerås,Poseidongatan,17,Öster Mälarstrand,72358,59.604871,16.571485,2.0,57,6,None,2020.0,3699.0,2637000.0
201,/annons/3734183,Lägenhet,Västerås,Stora gatan,1D,Centrum,72212,59.611443,16.554942,4.0,128,3,None,2008.0,7730.0,2795000.0
295,/annons/3893744,Lägenhet,Västerås,Välplanerad Etta - Stentorpsgatan,24A,Malmaberg,72343,59.624435,16.591758,1.0,26,1,None,1956.0,1911.0,595000.0
334,/annons/3785800,Lägenhet,Uppsala,None,None,Kvarngärdet,75425,59.873642,17.640564,2.0,53,1,None,2022.0,3350.0,1895000.0
464,/annons/3851345,Lägenhet,Uppsala,Wivalliusgatan,47,Nyby,75442,59.886590,17.656200,4.0,87,1,None,NaN,3920.0,1895000.0
474,/bostad/224045,Lägenhet,Uppsala,Petterslundsgatan,13,Fålhagen,75328,59.864015,17.654254,4.0,93,3,None,1963.0,3961.0,NaN
478,/annons/3846058,Lägenhet,Uppsala,Åskmolnsvägen,93,Storvreta,74335,59.950016,17.695329,3.0,77,1,None,1992.0,5581.0,NaN
479,/annons/3846064,Lägenhet,Uppsala,None,None,Kåbo,75645,59.835191,17.632985,2.0,60,4,None,NaN,3458.0,2200000.0
480,/annons/3846073,Lägenhet,Uppsala,None,None,Kåbo,75645,59.835191,17.632985,2.0,57,4,None,NaN,3458.0,2150000.0


We can see that all types except apartments have fairly high percentages of missing values. Let us assume that accomodations with null-valued housing societies will have the same housing society as the most frequent hosuing society for that zip code

In [35]:
#merge null-valued hs-rows with non-null valued hs on postal_code
df_accomodations_without_hs = df[(df["housing_society"].isnull())]
df_accomodations_with_hs = df[(df["housing_society"].notnull())]

df_merged = pd.merge(df_accomodations_without_hs,
                 df_accomodations_with_hs[['housing_society', 'postal_code']],
                 on='postal_code')

In [36]:
df_merged

,id,type,city,street,street_num,district,postal_code,latitude,longitude,rooms,square_meters,floor,housing_society_x,construction_year,fee,price,housing_society_y
0,/bostad/1846982,Villa,Helsingborg,Rusthållsgatan,5A,Miatorp,25231,56.015764,12.725562,5.0,112,0,None,1985.0,NaN,3295000.0,HSB BRF Palsternackan
1,/bostad/1846982,Villa,Helsingborg,Rusthållsgatan,5A,Miatorp,25231,56.015764,12.725562,5.0,112,0,None,1985.0,NaN,3295000.0,BRF Palsternackan 1
2,/bostad/1846982,Villa,Helsingborg,Rusthållsgatan,5A,Miatorp,25231,56.015764,12.725562,5.0,112,0,None,1985.0,NaN,3295000.0,HSB BRF Melonen
3,/bostad/1846982,Villa,Helsingborg,Rusthållsgatan,5A,Miatorp,25231,56.015764,12.725562,5.0,112,0,None,1985.0,NaN,3295000.0,HSB BRF Palsternackan
4,/bostad/2247046,Villa,Helsingborg,Grännagatan,7,Fältabacken,25250,56.039595,12.727825,4.0,95,0,None,1924.0,NaN,2995000.0,Riksbyggen BRF Helsingborgshus Nr 4
5,/bostad/2247046,Villa,Helsingborg,Grännagatan,7,Fältabacken,25250,56.039595,12.727825,4.0,95,0,None,1924.0,NaN,2995000.0,BRF Karlavagnen 3
6,/bostad/2247046,Villa,Helsingborg,Grännagatan,7,Fältabacken,25250,56.039595,12.727825,4.0,95,0,None,1924.0,NaN,2995000.0,HSB BRF Vanja
7,/bostad/2247046,Villa,Helsingborg,Grännagatan,7,Fältabacken,25250,56.039595,12.727825,4.0,95,0,None,1924.0,NaN,2995000.0,HSB BRF Vanja
8,/bostad/2242476,Villa,Helsingborg,Alvägen,13,Laröd,25482,56.093354,12.647752,8.0,161,0,None,1965.0,NaN,7475000.0,BRF Johannishus
9,/bostad/1882367,Villa,Helsingborg,Skolvägen,20,Laröd,25482,56.093584,12.650555,5.0,103,0,None,1958.0,NaN,5950000.0,BRF Johannishus


In [44]:
#Extract one of the most frequent housing societies for that zip-code
df_hs_mode = df_merged.groupby(df_merged.id).housing_society_y.agg(
    lambda x: pd.Series.mode(x)[0]).to_frame().reset_index().rename(columns={'housing_society_y':'housing_society'})

In [45]:
df_hs_mode.head(5)

,id,housing_society
0,/annons/2336989,Riksbyggen BRF Blå Staden
1,/annons/2404586,Riksbyggen BRF Blå Staden
2,/annons/3037177,BRF Sverigehusets Kubik 3
3,/annons/3037486,BRF Flygledaren
4,/annons/3074547,BRF Kikaren


In [46]:
df.set_index('id', inplace=True)
df.update(df_hs_mode.set_index('id'))
df.reset_index(inplace=True)
df

,id,type,city,street,street_num,district,postal_code,latitude,longitude,rooms,square_meters,floor,housing_society,construction_year,fee,price
0,/annons/3848254,Lägenhet,Uppsala,Torngatan,24,Kapellgärdet,75423,59.866360,17.638699,2.0,55,1,BRF Tornet 1,2011.0,3126.0,2095000.0
1,/bostad/280424,Lägenhet,Helsingborg,Lilla Bergaliden,1,Centrum,25223,56.044865,12.700500,2.0,74,2,BRF Minerva 16,1939.0,5028.0,2175000.0
2,/bostad/285988,Lägenhet,Helsingborg,Skaragatan,27,Rosengården,25263,56.050858,12.729998,2.0,45,3,Riksbyggen BRF Helsingborgshus nr 21,1978.0,2843.0,825000.0
3,/bostad/3906240,Lägenhet,Helsingborg,Kristinelundsgatan,11,Miatorp,25230,56.016640,12.721736,1.0,45,3,HSB BRF Melonen,NaN,3399.0,800000.0
4,/annons/3876994,Radhus,Helsingborg,Sommargatan,31,Västergård,25665,56.046673,12.750883,3.0,78,0,BRF Snövit,1993.0,5589.0,1595000.0
5,/bostad/792134,Lägenhet,Helsingborg,Föreningsgatan,30,Tågaborg,25441,56.056015,12.695240,2.5,79,3,BRF Föreningsgatan 28 och 30,1935.0,4593.0,2250000.0
6,/bostad/1846982,Villa,Helsingborg,Rusthållsgatan,5A,Miatorp,25231,56.015764,12.725562,5.0,112,0,HSB BRF Palsternackan,1985.0,NaN,3295000.0
7,/bostad/2247046,Villa,Helsingborg,Grännagatan,7,Fältabacken,25250,56.039595,12.727825,4.0,95,0,HSB BRF Vanja,1924.0,NaN,2995000.0
8,/annons/3898060,Lägenhet,Helsingborg,Michael Löfmans gata,11,Tågaborg,25438,56.053892,12.693812,2.0,64,3,BRF Falken 3,NaN,3488.0,2250000.0
9,/annons/3898172,Lägenhet,Helsingborg,Kullagatan,43,Centrum,25220,56.049209,12.691370,4.0,98,2,BRF Fiskaren i Helsingborg,1958.0,5108.0,2400000.0


In [47]:
df[df.housing_society.isnull()]

,id,type,city,street,street_num,district,postal_code,latitude,longitude,rooms,square_meters,floor,housing_society,construction_year,fee,price
33,/annons/3887215,Villa,Västerås,Fältspatsvägen,1,Munga,72594,59.742667,16.526091,2.0,55,0,None,1973.0,NaN,1995000.0
36,/bostad/2073765,Villa,Västerås,Malörtsvägen,16,Örtagården,72591,59.564192,16.465578,6.0,170,0,None,2003.0,NaN,5850000.0
37,/annons/3891158,Parhus,Helsingborg,Rååvägen,60B,Råå,25270,56.001603,12.737404,5.0,177,0,None,2020.0,NaN,6495000.0
51,/bostad/2815284,Villa,Helsingborg,Långgatan,17,Alla Bilder,25270,55.997960,12.739719,3.0,80,0,None,NaN,NaN,3475000.0
52,/annons/3877132,Villa,Helsingborg,Lövstigen,1,Miatorp,25241,56.023710,12.711099,7.0,167,0,None,1923.0,NaN,3995000.0
79,/bostad/1827908,Villa,Helsingborg,Prästkragegatan,15,Gustavslund,25667,56.033691,12.755810,6.0,194,0,None,2013.0,NaN,7975000.0
87,/bostad/1848983,Radhus,Västerås,Råbocksvägen,8,Gryta,72242,59.639379,16.535551,5.0,132,0,None,1978.0,NaN,2995000.0
89,/annons/3441795,Fritidshus,Västerås,Lönnbacksvägen,3,Gångholmen,72591,59.525045,16.526348,3.0,65,0,None,1945.0,NaN,995000.0
91,/bostad/2090288,Villa,Västerås,Tallvägen,16,Hökåsen,72231,59.658032,16.588499,8.0,109,0,None,1965.0,NaN,3695000.0
103,/annons/3773854,Villa,Västerås,Sevalla Väsby,6:5,Sevalla,72595,59.762512,16.719589,5.0,123,0,None,NaN,NaN,2900000.0


In [48]:
#num of null-values per type
df.housing_society.isnull().groupby(df["type"]).sum()

type
Fritidshus     86.0
Gård            9.0
Kedjehus       27.0
Lägenhet       35.0
Parhus         13.0
Radhus         51.0
Villa         292.0
Name: housing_society, dtype: float64

In [49]:
#percentage of null-values per type
df.housing_society.isnull().groupby(df["type"]).mean() * 100

type
Fritidshus     82.692308
Gård          100.000000
Kedjehus       67.500000
Lägenhet        0.584307
Parhus         24.528302
Radhus         26.701571
Villa          55.725191
Name: housing_society, dtype: float64

In [50]:
df.housing_society.isnull().sum()

513

We still have around 513 null values in *housing_society*. We could, for each *type*, check the most common housing society in that (1) district and (2) city and assign that value to the accomodation

In [51]:
df.groupby(['type', 'city', 'district'])['housing_society'].unique()

type        city         district                
Fritidshus  Göteborg     Askim                                                [None]
                         Fagerdahl                             [BRF Västra Nidingen]
                         Hisingen                              [BRF Västra Nidingen]
                         Hisings Kärra                         [BRF Västra Nidingen]
                         Kortedala                                            [None]
                         Lilleby                                              [None]
                         Linnarhult                                           [None]
                         Sillviks Havskoloni                                  [None]
                         Säve                                                 [None]
                         Torslanda Havsbadskoloni                  [BRF Flygledaren]
                         Västra Centrum                        [BRF Godhemsstjärnan]
               

# Longitude and Latitude

We know from earlier that there are no null-values in city, longitude and latitude. Knowing this, we can create a new feature which measures the distance between the accomodation and the city center using the haversine distance formula 

In [86]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [99]:
# extract all cities

cities = pd.DataFrame(df['city'].unique().tolist(), columns=['city'])
cities

,city
0,Uppsala
1,Helsingborg
2,Västerås
3,Linköping
4,Örebro
5,Malmö
6,Stockholm
7,Göteborg


In [100]:
#Get longitude and latitude for each city
geolocator = Nominatim(user_agent='DataExploration')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
cities['loc'] = cities['city'].apply(geocode)
cities['longitude'] = cities['loc'].apply(lambda loc: loc.longitude)
cities['latitude'] = cities['loc'].apply(lambda loc: loc.latitude)
cities.drop('loc', axis='columns', inplace=True)

In [101]:
cities

,city,longitude,latitude
0,Uppsala,17.638744,59.858613
1,Helsingborg,12.704068,56.044198
2,Västerås,16.546368,59.611099
3,Linköping,15.624525,58.409813
4,Örebro,15.215118,59.274729
5,Malmö,13.000157,55.605293
6,Stockholm,18.071093,59.325117
7,Göteborg,11.967017,57.707233


In [ ]:
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

In [67]:
def get_distance_to_city_center(df_row):
    geolocator = Nominatim(user_agent='DataExploration')
    
    

In [71]:
get_distance_to_city_center(df.loc[0,:])

(12.7040684, 56.0441984)

In [21]:
con.close()